<a href="https://colab.research.google.com/github/componavt/neural_synset/blob/master/src/dataset/wikt_labels_tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading labels with definitions

Source code: [Loading a custom dataset](https://colab.research.google.com/github/huggingface/notebooks/blob/master/course/videos/load_custom_dataset.ipynb#scrollTo=D2ekPOyykZDq), [video](https://www.youtube.com/watch?v=HyQgpJTkRdE).

Video: [The pipeline function](https://www.youtube.com/watch?v=tiZFewofSLM).

Install the Transformers and Datasets libraries to run this notebook.

In [ ]:
! pip install -U accelerate
! pip install -U transformers

! pip install datasets transformers[sentencepiece]
! pip install torch               # required by TrainingArguments
! pip install transformers[torch] # required by TrainingArguments

In [ ]:
!wget https://github.com/componavt/neural_synset/raw/master/data/label_meaning.csv

In [3]:
cat label_meaning.csv

"word"|"meaning"|"книжн."|"ирон."|"религ."|"груб."
подвизаться|осуществлять деятельность, работать, действовать в какой-нибудь области|1|1|0|0
подвизаться|совершать подвиг в чём-либо, часто о ежедневном борении|0|0|1|0
заткнуться|то же, что замолчать; перестать говорить, кричать, плакать; замолкнуть|0|0|0|1
пустобрёх|тот, кто говорит много пустого и несерьёзного; болтун|0|0|0|1
излаять|сильно изругать|0|0|0|1
бизнес-дама|о предпринимательнице|0|1|0|0
агнец божий|кроткий, робкий, безобидный человек|0|1|0|0
всезнайка|человек, который считает себя знающим всё|0|1|0|0
галантерейный|относящийся к галантерее|0|0|0|0
галантерейный|чрезмерно любезный, вежливый до слащавости|0|1|0|0
дитятя|дитя, ребёнок, чадо|0|1|0|0


In [4]:
from datasets import load_dataset

ds = load_dataset("csv", data_files="label_meaning.csv", sep="|")  # , split='train')
print(ds["train"][0]["word"], "=", ds["train"][0]["meaning"], )

Generating train split: 0 examples [00:00, ? examples/s]

подвизаться = осуществлять деятельность, работать, действовать в какой-нибудь области


In [ ]:
# Don't uncomment the following line unless your dataset is small!
# training_corpus = [ds["train"][i: i + 1000]["meaning"] for i in range(0, len(ds["train"]), 1000)]
# training_corpus

In [ ]:
# create corpus of strings, see https://huggingface.co/learn/nlp-course/chapter6/2 (Training a new tokenizer from an old one)
#training_corpus = (
#    ds["train"][i : i + 1000]["meaning"]
#    for i in range(0, len(ds["train"]), 1000)
#)
#training_corpus

In [23]:
def get_training_corpus(dataset):
    d = dataset["train"]
    for i in range(0, len(d), 1000):
        samples = d[i : i + 1000]
        yield samples["meaning"]
training_corpus = get_training_corpus(ds)

In [24]:
#def meaning_iterator():
#    for i in range(0, len(da["train"]), 1):
#        yield da["train"][i]["meaning"]
#print(len(datushka["train"]))
print(list(training_corpus))

[['осуществлять деятельность, работать, действовать в какой-нибудь области', 'совершать подвиг в чём-либо, часто о ежедневном борении', 'то же, что замолчать; перестать говорить, кричать, плакать; замолкнуть', 'тот, кто говорит много пустого и несерьёзного; болтун', 'сильно изругать', 'о предпринимательнице', 'кроткий, робкий, безобидный человек', 'человек, который считает себя знающим всё', 'относящийся к галантерее', 'чрезмерно любезный, вежливый до слащавости', 'дитя, ребёнок, чадо']]


# AutoTokenizer and PyTorch optimized training loop
From [quicktour.ipynb#AutoTokenizer](https://colab.research.google.com/github/huggingface/notebooks/blob/main/transformers_doc/en/quicktour.ipynb#scrollTo=c-mB_1hXw57y&line=1&uniqifier=1)

In [25]:
# setup hyperparameters (learning rate, batch size, and the number of epochs to train for)
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./pt_training",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    )

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
model_name2 = "Babelscape/wikineural-multilingual-ner"     # see https://huggingface.co/Babelscape/wikineural-multilingual-ner
model_name3 = "orgcatorg/bert-base-multilingual-cased-ner" # see https://huggingface.co/orgcatorg/bert-base-multilingual-cased-ner
pt_model    = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer   = AutoTokenizer.from_pretrained(model_name)
tokenizer2  = AutoTokenizer.from_pretrained(model_name2)
tokenizer3  = AutoTokenizer.from_pretrained(model_name3)

In [28]:
tokenizer_new  = tokenizer.train_new_from_iterator(training_corpus, 52000)
tokenizer_new2 = tokenizer2.train_new_from_iterator(training_corpus, 52000)
tokenizer_new3 = tokenizer3.train_new_from_iterator(training_corpus, 52000)

In [ ]:
# ds["train"][0]["meaning"]

In [29]:
s = ds["train"][0]["meaning"]
print(s)
#tokens_old = old_tokenizer.tokenize(s)
print(tokenizer(s),  "tokens by MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")
print(tokenizer2(s), "tokens by Babelscape/wikineural-multilingual-ner")     # subword tokenization
print(tokenizer3(s), "tokens by orgcatorg/bert-base-multilingual-cased-ner") # subword tokenization

#tokens = tokenizer.tokenize( ds["train"][0]["meaning"] )
#print("tokens=", tokens)

осуществлять деятельность, работать, действовать в какой-нибудь области
{'input_ids': [1, 24527, 22857, 73608, 834, 262, 260, 28877, 262, 19384, 16877, 316, 1067, 818, 265, 669, 14320, 9407, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]} tokens by MoritzLaurer/mDeBERTa-v3-base-mnli-xnli
{'input_ids': [101, 85854, 10227, 92157, 32589, 34112, 117, 45680, 117, 105805, 76183, 543, 78351, 118, 19544, 91369, 11431, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]} tokens by Babelscape/wikineural-multilingual-ner
{'input_ids': [101, 85854, 10227, 92157, 32589, 34112, 117, 45680, 117, 105805, 76183, 543, 78351, 118, 19544, 91369, 11431, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [30]:
def tokenize_dataset(dataset):
  return tokenizer(dataset["meaning"])

In [ ]:
ds["train"]
#ds

In [ ]:
#dataset = ds["train"].map(tokenize_dataset, batched=True)
dataset = ds.map(tokenize_dataset, batched=True)
dataset

In [ ]:
# create a batch of examples from dataset
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

In [ ]:
# 80% train, 20% test + validation
dataset = dataset['train'].train_test_split(test_size=0.2, shuffle=True)
dataset

In [35]:
# gather all these classes in Trainer:
from transformers import Trainer
trainer = Trainer(
    model=pt_model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)  # doctest: +SKIP

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

#see https://discuss.huggingface.co/t/the-model-did-not-return-a-loss-from-the-inputs-only-the-following-keys-logits-for-reference-the-inputs-it-received-are-input-values/25420